In [1]:
import pandas as pd
import requests
import re
import time
from io import StringIO
from curl_cffi import requests as cureq

In [2]:
import os

RAW_DIR = "/content/data_raw"
os.makedirs(RAW_DIR, exist_ok=True)


In [3]:
def fetch_fbref(url, sleep=3):
  response = cureq.get(url = url, impersonate = "chrome")
  time.sleep(sleep)
  return (response.text)

In [4]:

def inject_commented_tables(html: str, must_contain=("stats_standard",)): #FBref hides the tables in comments instead of common table tags
    comments = re.findall(r"<!--(.*?)-->", html, flags=re.S) #looks for comments
    blocks = []
    for c in comments: #search for all comments
        if "<table" in c and all(m in c for m in must_contain): #stats_standard is their way of labelling their tables for standard metrics
            blocks.append(c)
    return html + "\n".join(blocks), blocks #html fetched tables and hidden tables

#html2, blocks = inject_commented_tables(html, must_contain=("stats_standard",))
#print("bloques comentados con la tabla:", len(blocks))

#tables2 = pd.read_html(StringIO(html2))
#print("tablas totales ahora:", len(tables2))

#tables2[2].head()

In [9]:
def is_player_stats_table(df: pd.DataFrame) -> bool:
    if not isinstance(df.columns, pd.MultiIndex):
        return False

    # Tomamos el segundo nivel del MultiIndex (los nombres reales)
    cols = set(str(c).lower() for c in df.columns.get_level_values(1))

    required = {"player", "nation", "pos", "born"}

    return required.issubset(cols)

In [ ]:
    url = f"https://fbref.com/en/comps/41/2025/stats/2025-Primera-A-Stats"
    html = fetch_fbref(url)
    html2, blocks = inject_commented_tables(html, must_contain=("stats_standard",))
    tables = pd.read_html(StringIO(html2))
    print(is_player_stats_table(tables[2]))
    #for i, t in enumerate(tables):
      #print(i, t.columns)
    player_stats = tables[2].copy()

True


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [10]:
def fetch_tables(years, sleep=3):
  RAW_DIR = "/content/data_raw"
  os.makedirs(RAW_DIR, exist_ok=True)

  data_by_year = {}
  for year in years:
    url = f"https://fbref.com/en/comps/41/{year}/stats/{year}-Primera-A-Stats"
    try:
      html = fetch_fbref(url)
    except Exception as e:
      print(f"Error fetching {year}: {e}")
      continue
    html2, blocks = inject_commented_tables(html, must_contain=("stats_standard",))
    tables = pd.read_html(StringIO(html2))
    player_table = None
    for t in tables:
      if is_player_stats_table(t):
        player_table = t.copy()
        break
    if player_table is None:
      print(f"Player table not found for {year}")

    filename = f"FPC_player_stats_{year}_raw.csv"
    filepath = os.path.join(RAW_DIR, filename)
    player_table.to_csv(filepath, index=False)
    print(f"Saved {filename}")

    data_by_year[year] = player_table

    time.sleep(sleep)


In [11]:
def flatten_multiindex_join_levels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    lvl0 = df.columns.get_level_values(0)
    lvl1 = df.columns.get_level_values(1)

    new_cols = []
    for a, b in zip(lvl0, lvl1):
        if str(a).startswith("Unnamed"):
            a = ""
        else:
            a = str(a).strip().replace(" ", "_")

        b = str(b).strip().replace(" ", "_")

        new_cols.append(f"{a}_{b}" if a else b)

    df.columns = new_cols
    return df

In [19]:
def clean_player_stats(df_raw: pd.DataFrame, season: int) -> pd.DataFrame:
  df = df_raw.copy()

  df = flatten_multiindex_join_levels(df)


    #mapping columns into a better convention
  rename_map = {
        "Player": "player",
        "Nation": "nation",
        "Pos": "position",
        "Squad": "team",
        "Age": "age",
        "Born": "born",
        "Playing_Time_MP": "matches_played",
        "Playing_Time_Starts": "starts",
        "Playing_Time_Min": "minutes",
        "Playing_Time_90s": "nineties",
        "Performance_Gls": "goals",
        "Performance_Ast": "assists",
        "Per_90_Minutes_Gls": "goals_per_90",
        "Per_90_Minutes_Ast": "assists_per_90",
    }

  df = df.rename(columns=rename_map)
  #formatting columns to a numerical format
  numeric_cols = [
        "age", "born", "matches_played", "starts", "minutes", "nineties",
        "goals", "assists", "goals_per_90", "assists_per_90"
  ]

  for c in numeric_cols:
    if c in df.columns:
      df[c] = pd.to_numeric(df[c], errors="coerce")

  df["season"] = season
  print(df.columns.tolist())
  df = df[df["minutes"].fillna(0)>0]

  final_cols = [
        "player",
        "team",
        "position",
        "age",
        "matches_played",
        "minutes",
        "goals",
        "goals_per_90",
        "Performance_G-PK",
        "season"
    ]

  df = df[[c for c in final_cols if c in df.columns]]
  return df



In [20]:
years = list(range(2025, 2014, -1))
fetch_tables(years)

Saved FPC_player_stats_2025_raw.csv
Saved FPC_player_stats_2024_raw.csv
Saved FPC_player_stats_2023_raw.csv
Saved FPC_player_stats_2022_raw.csv
Saved FPC_player_stats_2021_raw.csv
Saved FPC_player_stats_2020_raw.csv
Saved FPC_player_stats_2019_raw.csv
Saved FPC_player_stats_2018_raw.csv
Saved FPC_player_stats_2017_raw.csv
Saved FPC_player_stats_2016_raw.csv
Saved FPC_player_stats_2015_raw.csv


In [14]:
RAW_DIR = "/content/data_raw"

files = sorted([
    f for f in os.listdir(RAW_DIR)
    if f.endswith("_raw.csv")
])

files


['FPC_player_stats_2015_raw.csv',
 'FPC_player_stats_2016_raw.csv',
 'FPC_player_stats_2017_raw.csv',
 'FPC_player_stats_2018_raw.csv',
 'FPC_player_stats_2019_raw.csv',
 'FPC_player_stats_2020_raw.csv',
 'FPC_player_stats_2021_raw.csv',
 'FPC_player_stats_2022_raw.csv',
 'FPC_player_stats_2023_raw.csv',
 'FPC_player_stats_2024_raw.csv',
 'FPC_player_stats_2025_raw.csv']

In [21]:
raw_dfs = {}

for f in files:
    year = int(f.split("_")[3])  # FPC_player_stats_2015_raw.csv
    path = os.path.join(RAW_DIR, f)
    raw_dfs[year] = pd.read_csv(path, header=[0,1])





In [16]:
test_df = clean_player_stats(raw_dfs[2025], season=2025)
print(type(test_df))
test_df.head()

['Rk', 'player', 'nation', 'position', 'team', 'age', 'born', 'matches_played', 'starts', 'minutes', 'nineties', 'goals', 'assists', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR', 'goals_per_90', 'assists_per_90', 'Per_90_Minutes_G+A', 'Per_90_Minutes_G-PK', 'Per_90_Minutes_G+A-PK', 'Matches', 'season']
<class 'pandas.core.frame.DataFrame'>


,Rk,player,nation,position,team,age,born,matches_played,starts,minutes,...,Performance_PKatt,Performance_CrdY,Performance_CrdR,goals_per_90,assists_per_90,Per_90_Minutes_G+A,Per_90_Minutes_G-PK,Per_90_Minutes_G+A-PK,Matches,season
0,1,José Abad Cuenú,co COL,DF,Junior,29.0,1995.0,18.0,15.0,1344.0,...,0,7,1,0.0,0.00,0.00,0.00,0.00,Matches,2025
1,2,Joao Abonia,co COL,"DF,MF",La Equidad,24.0,2000.0,33.0,29.0,2520.0,...,0,7,1,0.0,0.04,0.04,0.00,0.04,Matches,2025
2,3,Humberto Acevedo,co COL,GK,Llaneros,27.0,1997.0,2.0,2.0,180.0,...,0,0,0,0.0,0.00,0.00,0.00,0.00,Matches,2025
3,4,Felipe Acosta,co COL,MF,La Equidad,30.0,1994.0,19.0,16.0,1414.0,...,0,5,0,0.0,0.00,0.00,0.00,0.00,Matches,2025
4,5,Sebastián Acosta,co COL,MF,Pereira,30.0,1994.0,3.0,0.0,15.0,...,0,0,0,0.0,0.00,0.00,0.00,0.00,Matches,2025


In [24]:
clean_dfs = {}

for year, df_raw in raw_dfs.items():
  df_clean = clean_player_stats(df_raw, season=year)
  df_clean.to_csv(
        f"/content/data_clean/FPC_player_stats_{year}_clean.csv",
        index=False
  )

  clean_dfs[year] = df_clean


#

['Rk', 'player', 'nation', 'position', 'team', 'age', 'born', 'matches_played', 'starts', 'minutes', 'nineties', 'goals', 'assists', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR', 'goals_per_90', 'assists_per_90', 'Per_90_Minutes_G+A', 'Per_90_Minutes_G-PK', 'Per_90_Minutes_G+A-PK', 'Matches', 'season']
['Rk', 'player', 'nation', 'position', 'team', 'age', 'born', 'matches_played', 'starts', 'minutes', 'nineties', 'goals', 'assists', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR', 'goals_per_90', 'assists_per_90', 'Per_90_Minutes_G+A', 'Per_90_Minutes_G-PK', 'Per_90_Minutes_G+A-PK', 'Matches', 'season']
['Rk', 'player', 'nation', 'position', 'team', 'age', 'born', 'matches_played', 'starts', 'minutes', 'nineties', 'goals', 'assists', 'Performance_G+A', 'Performance_G-PK', 'Performance_PK', 'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR

In [23]:
def top_scorers(clean_dfs: dict, top_n: int = 15):
  top_all = []
  for year, df in clean_dfs.items():
    if "goals" not in df.columns or "minutes" not in df.columns:
      print(f"Not found goals on {year}'s csv")
      continue
    top_df = (
        df.assign(
            goal_rank = df["goals"].rank(method = 'dense', ascending = False)
        )
        .query("goal_rank <= @top_n")
        .sort_values(["goal_rank", "minutes"], ascending=[True, False])
    )
    top_all.append(top_df)

  top_all_df = pd.concat(top_all, ignore_index=True)
  filename = f"Top{top_n}_FPC_all_seasons.csv"
  path = f"/content/data_ready/{filename}"
  top_all_df.to_csv(
        path,
        index=False
    )

  print(f"Saved {filename}")

In [25]:
top_scorers(clean_dfs)


Saved Top15_FPC_all_seasons.csv
